# Physics 129 Fall 2025 Homework 3 

**Due Friday September 26 11:59PM**

submit the PDF version of the complete notebook to GradeScope

## Question 1: Fitting a Signal in the Presence of Background (30 points)

### Learning objectives
In this question you will:

- Gain experience in performing $\chi^2$ fits to histogrammed data; explore how the statistical significance of a signal depends on the number of signal events and the signal-to-background ratio


### 1a. 

Physicists often fit for signals in the presence of background.  In such cases, the significance of the fitted signal depends not only on the number of signal events but also on the amount of background and our ability to statistically separate the two.  In this problem, we will explore fitting signal and background for a very simple case:  The signal is a Gaussian peak centered at $x=10$ with a width $\sigma=1$ and the background is uniformly distributed between $x=0$ and $x=20$.  The code below generates fake data, allowing you to change both the number of events in the signal and the ratio of signal-to-background.  To make sure that our definition of background does not depend on the fit range, the code below defines the signal-to-background ratio as the ratio number of signal events to the number of background events in a $\pm 2 \sigma$ window around the signal peak.  Here is the code you will use to generate the fake data:

In [ ]:
# Write import math
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.optimize import minimize

def makeData( SignalToBackground, nSigEvents, seed=12345):
    """Generates a dataset consisting of nEvents some of which are signal (a Gaussian with a width of 1 centered
    at x=5) and the rest of which is background
    Definition of SignalToBackground:  The ratio of the number of signal events within 2 sigma of the peak to the number
    of background events in that same x-range  (this definition is just a choice)
    
    Parameters
    ==========
     SignalToBackground : float
     Ratio of the number of signal events within 2 sigma of the peak to the number
    of background events in that same x-range  (this definition is just a choice) 
      
    nSigEvents : int
      total number of signal events generated (NOT the number in +- 2 sigma)
      
    seed : int
      seed for the random number generator
      
    Returns
    =======
    data : array
      the measurements of x
    """
    fracOutsideTwoSigma = 4.55e-2
    # 1-fracOutsideTwoSigma is the fraction of the Signal Events within +-2 sigma
    # To get the total number of background events, find the number in +-2 sigma
    # which is 4 units of x and then since the background is flat from 0 to 20
    # multiply by 20/4=5
    
    nBackground = 5*nSigEvents*(1-fracOutsideTwoSigma)/SignalToBackground
    nEvents = nSigEvents+nBackground
    fSig = nSigEvents/nEvents
    
    # Make an array to hold the data (ie the x measurements)
    data = []
    
    # set the random seed.  This will allow us to reproduce the results if we run again
    np.random.seed(seed)
 
    # Retrieve nEvents random numbers that will be used to pick which events are signal
    # and which are background
    n = int(nEvents)
    tests = np.random.uniform(0,1,n)
    bck = np.random.uniform(0,20,n)
    sig = np.random.normal(10,1,n)
    
    count = 0
    for test in tests:
        if(test<fSig):
            data.append(sig[count])
        else:
            data.append(bck[count])
        count+=1
    
    # Loop over the events and pick either signal or background and draw from the appropriate 
    # pdf in each case
    
    # return the data to the user
    return data
    

Below is a simple test to show you how to use this code.  Remember if you run this code multiple times you should change the seed each time (for example, you could increment the seed each time you call the function)

In [ ]:
mydata = []
#  make 10 signal events with a signalToBackground of 1 using random seed 123
mydata=makeData(1,20,123)
print("number of total data events: ",len(mydata))
print("data: ",mydata)

Generate a sample of 1000 signal events with signalToBackground=0.5 and make a histogram of your results. Make sure that the number of bins in your histogram is small enough that you have at least 10 entries per bin (so that it is reasonable to do a binned fit to the histogram). To make life a bit easier, here is function you can use to make your histograms.  You are of course free to write your own function and not use this code.

In [ ]:
#Import the pyplot module of matplotlib as "plt"
import matplotlib.pyplot as plt


#Makes a histogram filled with the random numbers we generate
def plot_histogram(samples,xtitle,ytitle, title, nbins, limits):
   
    #Plot the histogram of the sampled data with nbins and a nice color
    n, bins, patches =plt.hist(samples, bins=nbins, range=limits, color=(0,0.7,0.9))  #Set the color using (r,g,b) values or
                                                                  #  use a built-in matplotlib color""" 
    bincenters = 0.5*(bins[1:]+bins[:-1])
    errs = np.sqrt(n)

    plt.errorbar(bincenters, n, yerr=errs, fmt='none')
    #Add some axis labels and a descriptive title
    plt.xlabel(xtitle)
    plt.ylabel(ytitle)

    #Get rid of the extra white space on the left/right edges (you can delete these two lines without a problem)
    xmin, xmax, ymin, ymax = plt.axis()
    plt.axis([limits[0],limits[1],ymin,ymax])

    #Not necessarily needed in a Jupyter notebook, but it doesn't hurt
    plt.show()
    return n, bins, patches

In [ ]:
#Write your answer here

### 1b. 

Pretend this is real data.  Your goal is to find the best estimate of how many events are in a Gaussian peak with unknown mean and sigma and what the uncertainty on this estimate is.  In your fit, you can make the assumption that the background is flat (a zeroth order polynomial) but that you don't have a prediction for the background rate.  Use your favorite minimization package to fit the data.  Deterimine the best estimate of the number of events in signal and the uncertainty on that estimate.  (remember, that you must let the position and width of the Gaussian and the size of the background vary in your fit).

Hint: For examples of how to perform a least squared fit of a function to data see:
- https://github.com/berkeley-physics/Python-Tutorials/blob/master/3%20-%20Specific%20topics/Fitting.ipynb
- https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html

In [ ]:
#Write your answer here

### 1c. 

What is the $\chi^2$ per degree of freedom for your fit?  What does this number tell you about how well your fit describes the data?

In [ ]:
#Write your answer here

### 1d. 

If a signal of size $N_S^{meas}$ has a fitted uncertainty $\sigma_S$, the signficance of the measurement is defined to be
$$
S^{meas} \equiv \frac{N_S^{meas}}{\sigma_S}
$$
When the size of the data sample is large enough that Gaussian uncertainties are appropriate, a rule of thumb can be used to give a crude estimate of the expected significance $S^{expected}$ of the measurement. This predicted signficance depends on the number of signal events ($N_{S}$) and is the number of background events populating the region **beneath the signal** ($N_B$):
$$
S^{expected}  \approx  \frac{N_S}{\sqrt{N_S+N_B}}
$$

Repeat the above exercise changing both the number of events in your signal and the signalToBackground ratio.  Plot the values of the measured significance  $S^{meas}$ 
obtained from your fits as a function of $\frac{N_S}{\sqrt{N_S+N_B}}$.  How do your results compare to the simple rule of thumb quoted above?

In [ ]:
#Write your answer here

## Question 2: Noether's Theorem (20 points)

### Learning objectives
In this question you will:

- Review the meaning of Noether's theorem
- Apply Noether's theorm to a non-relativistic quantum mechanical example 

In non-relativistic quantum mechanics, we learned that if an operator commutes with the Hamiltonian, the expectation value of that operator is a conserved quantity.  The same concept holds in relativistic quantum mechanics but is expressed using Lagrangian language.  The Lagrangian can be expressed as $$ L = T - V$$ where $T$ is the kinetic energy of the system and $V$ is the potential energy of the system. The action, S, of a trajectory is defined as the integral of the Lagrangian with respect to time during the trajectory, i.e. $$ S = \int L \hspace{0.05in} dt $$.

Noether's theorem tells us that for every symmetry of the action there is a conserved quantum number.

Consider the following example for the case of non-relativisitic quantum mechanics.  A particle with spin-$\frac{1}{2}$ and magnetic moment $\vec \mu = g \frac{q}{2m} \vec S$ where $q$ is the charge of the particle and $\vec S$ is the spin is placed in a constant magnetic field in the $z$-direction $\vec B = B_0 \hat z$.  The Lagrangian is therefore
$$
L = \frac{\left ( \vec p \right ) ^2}{2m}  + \vec \mu \cdot B
$$
Which of the following are conserved quantities:
- (a) $p_x$, the $x$-component of the momentum
- (b) $p_z$, the $z$-component of the momentum
- (c) $S_x$, the $x$-component of the spin
- (d) $S_z$, the $z$-component of the spin
- (e) $\vec S^2$ the magnitude of the spin-squared operator

Write your answer here

## Question 3: Parity Properties of Various Operators (20 points)

### Learning objectives
In this question you will:

- Review the meaning of the terms vector, axial vector, scalar and pseudoscalar and determine the parity property of several operators

We saw in class that operators could be classified according to their properties under the parity operator.  Vector operators change sign under parity

$$
{\bf P}\; \vec r \; {\bf P}^\dagger \rightarrow - \vec r
$$

while axial vectors do not:

$$
{\bf P }\;\vec L \; {\bf P } ^\dagger \rightarrow + \vec L
$$

similarly, scalar operators do not change sign under parity 

$$
{\bf P}\; \left (\vec r \cdot \vec p \right ) \; {\bf P}^\dagger \rightarrow + \vec r \cdot \vec p
$$

while pseudoscalar operators do change sign:

$$
{\bf P}\; \left (\vec p \cdot \vec L \right ) \; {\bf P}^\dagger \rightarrow - \vec p \cdot \vec L 
$$

For each of the operators below, state whether they are scalar, pseudoscalar. vector or pseudovector:
- (a) $\vec p_1 \cdot \left (\vec p_2 \times \vec p_3 \right )$ where 1. 2 and 3 are three different particles
-(b)  $\vec p_1 \times \left (\vec p_2 \times \vec p_3 \right )$ where 1. 2 and 3 are three different particles
- (c) The magnetic moment $\vec \mu$ of a particle
- (d) The magnetic field $\vec B$\;\; (Hint: look at the Biot-Savart law)
- (e) $\left (\vec p_1 \times \vec p_2 \right) \cdot \left (\vec S_1+\vec S_2 \right)$ where 1 and 2 are two different particles


#Write your answer here

## Question 4: Parity in Particle Decays (10 points)

### Learning objectives
In this question you will:

- Explore how conservation laws can be used to exclude specific decay channels

Show that a scalar meson (a meson with spin 0 and parity +1) cannot decay to three pseudoscalar mesons (mesons with spin 0 and parity -1) in a parity-conserving process such as the strong interaction.

#Write your answer here

### AI Technology Disclosure Statement

Use of AI technology is **permissible only** for:  
- Formatting answers (e.g., converting to Markdown or LaTeX)  
- Assisting with coding tasks (e.g., syntax help, debugging)  

Students must explicitly state **whether AI technology was used** in completing this homework.  
- If **yes**, describe **how** it was used.  
- If **no**, state that no AI technology was used.  

⚠️ If this disclosure is missing, the homework **will not be graded**.  
If it is later completed, the submission will be treated as **late**.


#### Your Answer